In [2]:
from sage.schemes.elliptic_curves.constructor import EllipticCurve
from sage.arith.misc import random_prime, is_prime
from sage.rings.integer_ring import ZZ #ZZ

# from sage.rings.finite_rings.finite_field_constructor import GF #GF
from sage.rings.finite_rings.finite_field_constructor import GF
from sage.rings.infinity import PlusInfinity
from sage.rings.finite_rings.finite_field_prime_modn import FiniteField_prime_modn
from sage.rings.finite_rings.finite_field_pari_ffelt import FiniteFieldElement_pari_ffelt
from sage.rings.polynomial.polynomial_ring_constructor import PolynomialRing

In [3]:
from sage.schemes.elliptic_curves.ell_finite_field import EllipticCurve_finite_field
from sage.schemes.elliptic_curves.ell_point import EllipticCurvePoint_finite_field


In [4]:
from sage.rings import integer

In [5]:
# from sage.rings.infinity import PlusInfinity
# type(+oo)
# +oo

In [6]:
##############################################################
#Implementation of the Boneh-Franklin Identity Based Encryption using the Weil Pairing
#Florence Lam
#Accompanies a paper with the same title
#References to Theorems/Definitions/Algorithms/Sections are to that paper
##############################################################


#BasicIdent requires two hash functions (cf. Section 2.4)
#We use this library
from Crypto.Hash import SHA512

#Okay for Hash(ID)
def H1(hash_input) -> int:
    hash = SHA512.new()
    str_val = str(hash_input)
    byte_val = str_val.encode()
    hash.update(byte_val)
    hexadecimal = hash.hexdigest()
    return int(hexadecimal, 16)

In [7]:
#Fast modular exponentiation algorithm (needed for F_{p^2})
def fastPowerSmall(g,A):
    if g == 0 and A == 0:
        return "Undefined"
    else:
        a = g
        b = 1
        while A > 0:
            if A%2 == 1:
                b = b*a
            a = a^2
            A = A//2
            print
        return b

In [8]:
#ASCII implementation
def textToInt(w):
    n  = 0
    counter = 0
    #counter will give us the index of each character in the string
    for i in w:
        n = n + ord(i)*(256**counter)
        counter = counter + 1
    return n

def intToText(n):
    str = ""
    while n > 0:
        a0 = n%256
        str = str + chr(a0) #chr undoes ord. ord() inputs character and outputs integer. str inputs integer between 0 and 255 and outputs character.
        n = n//256 #This is the quotient after dividing n by 256
    return str

In [9]:
# First step of Miller's Algorithm
# This defines a rational function g(x,y) on E whose divisor is div(g) = [P] + [Q] - [P+Q] - [O]
# (cf. Theorem 2.2)

# A, B coefficients of E. Use A,B = E.a_invariants()[3], E.a_invariants()[4]
# P = E([xP, yP])


def g(
    P: EllipticCurvePoint_finite_field,
    Q: EllipticCurvePoint_finite_field,
    x: integer.Integer,
    y: integer.Integer,
    E: EllipticCurve_finite_field,
) -> FiniteFieldElement_pari_ffelt:
    A, B = E.a_invariants()[3], E.a_invariants()[4]
    if P == E(0) or Q == E(0):
        return "no divisor"
    xP, yP = P[0], P[1]
    xQ, yQ = Q[0], Q[1]
    # Calculate slope of line connecting P and Q
    # JUST check if equal
    if yP == -yQ and xP == xQ: 
        slope = PlusInfinity()  # symbol for Infinity
    elif P == Q:
        slope = (3 * (xP**2) + A) / (2 * yP)
    else:
        slope = (yQ - yP) / (xQ - xP)
    # return the function on E
    if slope == PlusInfinity():
        return x - xP
    else:
        return (y - yP - slope * (x - xP)) / (x + xP + xQ - slope**2)

In [10]:
# Implementation of Miller's algorithm (Theorem 2.2)


def MillerAlgorithm(
    P: EllipticCurvePoint_finite_field,
    m: integer.Integer,
    x: integer.Integer,
    y: integer.Integer,
    E: EllipticCurve_finite_field,
):
    A, B = E.a_invariants()[3], E.a_invariants()[4]
    xP, yP = P[0], P[1]
    binary = m.digits(2)  # gives number in binary
    n = len(binary)  # trying to find what "n" is.
    T = P
    f = 1
    for i in range(
        n - 2, -1, -1
    ):  # Stop once i = -1, so last number is 0...range(start, stop, step)
        f = (f**2) * g(T, T, x, y, E)
        T *= 2  # T = 2T
        if binary[i] == 1:
            f = f * g(T, P, x, y, E)
            T += P
    print(f"f:{type(f)}")
    return f  # This algorithm returns a value...

In [11]:
# Weil Pairing Implementation (cf. Theorem 2.3)
# Works on general elliptic curve over finite field


def WeilPairing(
    P: EllipticCurvePoint_finite_field,
    Q: EllipticCurvePoint_finite_field,
    m:integer.Integer,
    E: EllipticCurve_finite_field,
):
    A, B = E.a_invariants()[3], E.a_invariants()[4]
    S = E.random_element()
    while m * S == E(0):
        S = (
            E.random_element()
        )  # Pick point S that is not m-torsion. This guarantees that S isn't a linear combination of P and Q.
    xS, yS = S[0], S[1]
    QplusS = Q + S
    f_P_QplusS = MillerAlgorithm(P, m, QplusS[0], QplusS[1], E)
    f_P_S = MillerAlgorithm(P, m, xS, yS, E)
    num = f_P_QplusS / f_P_S

    PminusS = P - S
    f_Q_PminusS = MillerAlgorithm(Q, m, PminusS[0], PminusS[1], E)  # This is f_Q(P-S)
    f_Q_minusS = MillerAlgorithm(Q, m, xS, -yS, E)  # This is f_Q(-S)
    denom = f_Q_PminusS / f_Q_minusS

    e_m = num / denom
    return e_m

In [12]:
#Implementation of Modified Weil Pairing (cf. Definition 2.8)
#Works on the elliptic curve E given by y^2 = x^3 + 1 over F_p with p a prime congruent to 2 modulo 3

def ModifiedWeilPairing(
    P:EllipticCurvePoint_finite_field,
    Q:EllipticCurvePoint_finite_field,
    m:integer.Integer,
    E:EllipticCurve_finite_field
    ):
    # Fp = GF(p)
    # R.<x> = Fp[]
    # Fp2.<z> = Fp.extension(x^2+x+1)      #Form F_{p^2} by adjoining z = {a nontrivial cube root of 1}
    # 
    Fp:FiniteField_prime_modn = GF(p)
    # Define the polynomial ring over Fp
    R = PolynomialRing(Fp, 'x')
    x = R.gen()
    # Define the extension field Fp2
    Fp2 = Fp.extension(x**2 + x + 1, 'z')
    z = Fp2.gen()

    E_zeta: EllipticCurve_finite_field = EllipticCurve(Fp2, [0,1])   #Define E: y^2 = x^3 + 1 over this field
    phiQ = E_zeta([z*Q[0],Q[1]])
    A,B = E_zeta.a_invariants()[3], E_zeta.a_invariants()[4]
    P_zeta = E_zeta([P[0],P[1]])
    S = E_zeta.random_element()
    while m*S == E(0):
        S = E_zeta.random_element()
    xS,yS = S[0],S[1]
    QplusS = phiQ + S
    f_P_QplusS = MillerAlgorithm(P,m,QplusS[0],QplusS[1],E_zeta)
    f_P_S = MillerAlgorithm(P,m,xS,yS,E_zeta)
    num = f_P_QplusS/f_P_S

    PminusS = P_zeta - S #modify
    f_Q_PminusS = MillerAlgorithm(phiQ,m,PminusS[0],PminusS[1],E_zeta) #This is f_Q(P-S)
    f_Q_minusS = MillerAlgorithm(phiQ,m,xS,-yS,E_zeta) #This is f_Q(-S)
    denom = f_Q_PminusS/f_Q_minusS

    e_m = num/denom
    print(type(e_m))
    return e_m

In [13]:
# Implementation of BDHGenerator
# THis is `Setup` algorithm in IBE scheme ###2
# (cf. Algorithm 2 in Section 3.1)


def BDHGenerator(k):
    print((2 ^ k) - 1)
    q = random_prime(
        (2 ^ k) - 1, True, lbound=2 ^ (k - 1)
    )  # Generates a random k-bit prime. False means using pseudo-primality tests.
    p = q
    l = 1  # need l for `MapToPoint`
    lq = q
    while True:
        l += 1
        lq += q
        p = lq - 1
        if p % 3 == 2 and (p + 1) % (q ^ 2) != 0 and is_prime(p) == True:
            break
    E: EllipticCurve_finite_field = EllipticCurve(
        GF(p), [0, 1]
    )  # p is public b/c the elliptic curve is known
    P = E(0)
    while P == E(0):
        Q = E.random_element()
        while Q == E(0):  # make sure P is not O
            Q = E.random_element()
        h = (p + 1) // q  # This is to make sure P has order q.
        P = h * Q  # Order of P is q1
    s = ZZ.random_element(2, q - 1)  # s is private master key in Z_q*.
    P_pub = s * P
    params = [p, q, l, E, P, P_pub]
    return params, s  # Everything except s is `params`/public

In [14]:
#Implementation of MapToPoint
#(cf Algorithm 3 in Section 3.2)

#A brief sketch of how this is used:
#y0 in Fp. It is an output from hash function. Input to the first hash function is ID.
#s is the master key
#private key is d_ID (don't send to others)
#MapToPoint is `Extract` in Boneh-Franklin
#Q_ID is for people to use so they can encrypt. They can just use ID, hash it, and calculate Q_ID by themselves. One way is to make a `public` MapToPoint function that doesn't take in s, and doesn't d_ID
#Admin does this, not the user
#Hash y0 before inputting to MapToPoint. y0 = Hash(ID)
#y0 is an element mod p, need to reduce mod p.

def MapToPoint(y0,params, s):
    p, q, l, E, P, P_pub = params
    x0 = pow((y0^2) - 1, ((2*p)-1)//3, p) #`pow` is Python's built-in function that does fast power
    print(x0)
    print(y0)
    Q = E([x0,y0])
    Q_ID = l*Q #l comes from BDHGenerator. It's the integer s.t. p = lq-1
    d_ID = s*Q_ID
    return Q_ID,d_ID

In [15]:
###Encrypt and Decryptimplementation
#(cf. Section 2.4 and end of Section 3.2)

def Encrypt(M,Q_ID,params):
    p, q, l, E, P, P_pub = params
    r = ZZ.random_element(2,q-1)
    print("r", type(r), r)
    print("P", type(P), P)
    U = r*P
    g_ID = ModifiedWeilPairing(Q_ID,P_pub,q,E)
    g_ID_to_r = fastPowerSmall(g_ID, r)
    V = M.__xor__(H1(g_ID_to_r))
    print("V", type(V), V)
    C = [U,V] #This is the ciphertext
    return C

#^^ is XOR operator in Sage
#d_ID from MapToPoint. It's the private key (I keep it)

def Decrypt(C,d_ID,params):
    p, q, l, E, P, P_pub = params
    U,V = C
    weil = ModifiedWeilPairing(d_ID,U,q,E)
    M = V.__xor__(H1(weil))
    return M


In [16]:
#########################################
#What follows is an example runthrough of the implementation in 5 steps
#Follows the outline of a general IBE scheme (cf. Section 2.4) implemented using the modified Weil pairing
#The user is encouraged to run the code and experiment
#########################################

In [17]:
#Step 1: Only need to run once
#Boneh-Franklin Page 20 says to pick primes p that are at least 512-bit long for security purposes.
params,s = BDHGenerator(512)
p, q, l, E, P, P_pub = params
print("p = ", p)
print("q = ", q)
print("E = ", E)
print("P = ", P)
print("s = ", s)
print("l = ", l)
print("P_pub = ", P_pub)

13407807929942597099574024998205846127479365820592393377723561443721764030073546976801874298166903427690031858186486050853753882811946569946433649006084095
p =  35007979873574564249714324106682761489587031386787233113368763285014597986359056041495671860376383182111778959880283423505662229386178191267334323867290146369
q =  9973783439764833119576730514724433472816818058913741627740388400289059255372950439172556085577317145900791726461619208975972145124267290959354508224299187
E =  Elliptic Curve defined by y^2 = x^3 + 1 over Finite Field of size 35007979873574564249714324106682761489587031386787233113368763285014597986359056041495671860376383182111778959880283423505662229386178191267334323867290146369
P =  (34805360992635997485334297767435334672234731924533481847934001100953289734400535086078253737645648529440276083174858255254146709804729376633579353220203446161 : 120248939483812105489778320814894327528135761558002583861857794959974154713261100778487950386788151703987624202229973518767

In [18]:
#Step 2: Hash. Only need to run once for one ID.
your_ID = "user"

y0 = H1(your_ID)
print("y0 = ", y0)

y0 =  9284027244711476868736574711199185332133782967237044148153727596027875067278862165519633211530008452951448644427085930313537133830441524639977514866580706


In [19]:
#Step 3: MapToPoint. Only need to run once.
Q_ID,d_ID = MapToPoint(y0,params, s)
print("Q_ID = ", Q_ID)
print("d_ID = ", d_ID)

10957140820579684980396929999135370903781184892251794569270427349221020023570906177823634064931083865021723294125040794606773293179089560186626562779463324137
9284027244711476868736574711199185332133782967237044148153727596027875067278862165519633211530008452951448644427085930313537133830441524639977514866580706
Q_ID =  (19055927573583377605736047058463810197455892587578689700850333741885254364774535880064211229659386606512160735335713993980009278432617164803419868464254824714 : 19025421094798717421800385342600311002116304107394708645383956895544762350866877772585730437138224699396988779116153713671882430127136020120649732409356013142 : 1)
d_ID =  (6926947756944103424156036877701085525293927920769287951252490493002600957182087000876418751796977020210666519235832613932327778705297118964279977571530942108 : 6347547923112217557833884506875649897741045220518382674878550992773052260979840753317707172411051773053331943303412335388023714173135577908652506323635439985 : 1)


In [20]:
#Step 4: Encrypt
input_message = "Hello! :)"

M = textToInt(input_message)
C = Encrypt(M,Q_ID,params)
U,V = C
print("M = ", M)
print("U = ", U)
print("V = ", V)

r <class 'sage.rings.integer.Integer'> 709484080632605826050256449452033585767533672370136590367476970474101770695035947860655755454920387513127738427290095700962705290915187970972380843385614
P <class 'sage.schemes.elliptic_curves.ell_point.EllipticCurvePoint_finite_field'> (34805360992635997485334297767435334672234731924533481847934001100953289734400535086078253737645648529440276083174858255254146709804729376633579353220203446161 : 12024893948381210548977832081489432752813576155800258386185779495997415471326110077848795038678815170398762420222997351876713719345211391859201761683086509673 : 1)
f:<class 'sage.rings.finite_rings.element_pari_ffelt.FiniteFieldElement_pari_ffelt'>
f:<class 'sage.rings.finite_rings.element_pari_ffelt.FiniteFieldElement_pari_ffelt'>
f:<class 'sage.rings.finite_rings.element_pari_ffelt.FiniteFieldElement_pari_ffelt'>
f:<class 'sage.rings.finite_rings.element_pari_ffelt.FiniteFieldElement_pari_ffelt'>
<class 'sage.rings.finite_rings.element_pari_ffelt.FiniteF

In [21]:
Ub: EllipticCurvePoint_finite_field = U
Ub.xy()

(29319698473328783491581024285605467459039516982824810719403686520576641124646360028849380907393818896184026453772171892995132907448270781141018146079618350187,
 13847974992908137821425660052268307279124469313627581105734063753502505165041675562603231309989975356073307655075214694585196646146698802686075598324866383737)

In [29]:
EllipticCurvePoint_finite_field(E, (int(str(Ub.x())), int(str(Ub.y()))))

(29319698473328783491581024285605467459039516982824810719403686520576641124646360028849380907393818896184026453772171892995132907448270781141018146079618350187 : 13847974992908137821425660052268307279124469313627581105734063753502505165041675562603231309989975356073307655075214694585196646146698802686075598324866383737 : 1)

In [ ]:
int(str(Ub.x()))

29319698473328783491581024285605467459039516982824810719403686520576641124646360028849380907393818896184026453772171892995132907448270781141018146079618350187

In [24]:
E

Elliptic Curve defined by y^2 = x^3 + 1 over Finite Field of size 35007979873574564249714324106682761489587031386787233113368763285014597986359056041495671860376383182111778959880283423505662229386178191267334323867290146369

In [25]:
Ub.curve()

Elliptic Curve defined by y^2 = x^3 + 1 over Finite Field of size 35007979873574564249714324106682761489587031386787233113368763285014597986359056041495671860376383182111778959880283423505662229386178191267334323867290146369

In [26]:
U

(29319698473328783491581024285605467459039516982824810719403686520576641124646360028849380907393818896184026453772171892995132907448270781141018146079618350187 : 13847974992908137821425660052268307279124469313627581105734063753502505165041675562603231309989975356073307655075214694585196646146698802686075598324866383737 : 1)

In [27]:
EllipticCurvePoint_finite_field()

TypeError: EllipticCurvePoint_field.__init__() missing 2 required positional arguments: 'curve' and 'v'

In [ ]:
type(U[0])

<class 'sage.rings.finite_rings.integer_mod.IntegerMod_gmp'>

In [ ]:
type(V)

<class 'sage.rings.integer.Integer'>

In [ ]:
#Step 5: Decrypt
M0 = Decrypt(C,d_ID,params)
output_message = intToText(M0)
print("M0 = ", M0)
print("messsage = ", output_message)

f:<class 'sage.rings.finite_rings.element_pari_ffelt.FiniteFieldElement_pari_ffelt'>
f:<class 'sage.rings.finite_rings.element_pari_ffelt.FiniteFieldElement_pari_ffelt'>
f:<class 'sage.rings.finite_rings.element_pari_ffelt.FiniteFieldElement_pari_ffelt'>
f:<class 'sage.rings.finite_rings.element_pari_ffelt.FiniteFieldElement_pari_ffelt'>
<class 'sage.rings.finite_rings.element_pari_ffelt.FiniteFieldElement_pari_ffelt'>
M0 =  760504891437990307144
messsage =  Hello! :)


In [ ]:
type(p)

<class 'sage.rings.integer.Integer'>

In [ ]:
################
#Open Cell for the user to test and experiment with any and all functions above
################

In [ ]:
type(E)

<class 'sage.schemes.elliptic_curves.ell_finite_field.EllipticCurve_finite_field_with_category'>